# 2020-07-17
###### Created by Weber Huang
+ 鎖定第一輪與最末輪對話的最大標籤，預測其餘標籤(E.g. Round 01 : Fix GNUG0, predict CNaN)。
+ 更改class Attention 為 tf.keras.attention，並且加入模型。
+ attention + 針對每1輪的每一個標籤做關鍵字(LLR 提取關鍵字)。
+ 訓練各輪次時也要放入前1輪的對話內容。
+ 新增特徵feature: 給兩個維度，總共有幾輪，現在是第幾輪(E.g. [1,7],[2,7]...)。

In [1]:
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import numpy as np
from collections import Counter
from sklearn import preprocessing
import jieba
import re

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import concatenate, Dense, Embedding, Dropout, LSTM, Bidirectional
from tensorflow.keras import Model, Input
# from keras.layers import Convolution1D, Flatten, , MaxPool1D, GlobalAveragePooling1D
# from keras.layers import concatenate, Bidirectional
# from keras import initializers
# from keras.layers.core import Dense, Dropout, Activation, Flatten
# from keras.layers.embeddings import Embedding
# from keras.layers.recurrent import SimpleRNN
# from keras.layers.recurrent import LSTM
# from keras.activations import tanh, softmax
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Attention

C:\Users\doudi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\doudi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\doudi\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\doudi\AppData\Roaming\Python\Python37\site-

In [3]:
def str_to_list(dataframe, string): 
    l = []
    n = []
    for i in dataframe[string]:
        if i != '[]':
            
            l.append([float(j) for j in str(i).split()])
        else:
            l.append(n)
    return l

# calculate length of the texts
def length(dataframe):
    len_num = []
    for i in dataframe['texts']:
        temp = i.replace(" ","")
        len_num.append(len(temp))
    
    dataframe['length'] = len_num
    return dataframe

In [4]:
from scipy import stats
import tensorflow as tf

def normalize(pred, truth):
    """ convert inputs to np.array and make sure
    inputs are normalized probability distributions
    """
    if len(pred) != len(truth):
        raise ValueError("pred and truth have different lengths")
    if len(pred) == 0 or len(truth) == 0:
        raise ValueError("pred or truth are empty")

    pred, truth = np.asarray(pred), np.asarray(truth)
    if not ((pred >= 0).all() and (truth >= 0).all()):
        raise ValueError("probability distribution should not be negative")
    pred, truth = pred / pred.sum(), truth / truth.sum()
    return pred, truth

def jensen_shannon_div(pred, truth, base=2):
    ''' JSD: Jensen-Shannon Divergence
    '''
    pred, truth = normalize(pred, truth)
    m = 1. / 2 * (pred + truth)
    return (stats.entropy(pred, m, base=base)
            + stats.entropy(truth, m, base=base)) / 2.

def root_normalized_squared_error(pred, truth):
    """ RNSS: Root Normalised Sum of Squares
    """

    def squared_error(pred, truth):
        return ((pred - truth) ** 2).sum()

    pred, truth = normalize(pred, truth)
    return np.sqrt(squared_error(pred, truth) / 2)

def jsd_custom_loss(y_true, y_pred):
            
    # calculate loss, using y_pred
    ''' JSD: Jensen-Shannon Divergence
    '''
#     y_pred, y_true = normalize(y_pred, y_true)
    m = 1. / 2 * (y_pred + y_true)
    # loss = (stats.entropy(y_pred, m, base=2) + stats.entropy(y_true, m, base=2)) / 2.
    # tf.keras.losses.KLD()
    loss = (tf.keras.losses.KLD(y_pred, m) + tf.keras.losses.KLD(y_true, m)) / 2.
    return loss
  

### 1. 新增特徵feature

In [44]:
train = pd.read_csv('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\train_baidu_cn.csv', encoding='utf_8')
dev = pd.read_csv('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\dev_baidu_cn.csv', encoding='utf_8')

In [45]:
# test = pd.read_csv('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\test_w2v_cn.csv', encoding='utf_8')

In [19]:
len(dev)

1755

In [46]:
train['index'] = train.index
dev['index'] = dev.index
# test['index'] = test.index

In [47]:
def round_feature(dataframe):
    Id_list = dataframe['id'].unique()
    final = []
    # go through each Id first
    for Id in tqdm(Id_list):  
        for i in range(len(dataframe)):
            if dataframe['id'][i] == Id:
                length = len(dataframe[dataframe['id']==Id])
                final.append([dataframe['round'][i],length])
            else:
                continue
    return final

In [48]:
train['round_feature'] = round_feature(train)
dev['round_feature'] = round_feature(dev)
# test['round_feature'] = round_feature(test)

C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


### 2. 鎖定第一輪與最末輪(最末輪無法鎖定)對話的最大標籤

In [61]:
def find_head_tail(dataframe):
    Id_list = dataframe['id'].unique()
    final = []
    # go through each Id first
    for Id in tqdm(Id_list):  
        for i in range(len(dataframe)):
            if dataframe['id'][i] == Id:
                length = len(dataframe[dataframe['id']==Id])
                if dataframe['round'][i] == 1:
                    # case head
                    final.append(0)
                elif dataframe['round'][i] == length:
                    # case tail
                    final.append(2)
                else:
                    # None 
                    final.append(1)
            else:
                continue
    return final

In [62]:
train['head_tail'] = find_head_tail(train)
dev['head_tail'] = find_head_tail(dev)

C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


> load

In [8]:
train = pd.read_csv('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\train_feature.csv', encoding='utf_8')
dev = pd.read_csv('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\dev_feature.csv', encoding='utf_8')

In [9]:
train["W2V"] = str_to_list(train, 'W2V_str')
dev["W2V"] = str_to_list(dev, 'W2V_str')
# test["W2V"] = str_to_list(test, 'W2V_str')

In [7]:
# train = length(train)
# dev = length(dev)
# test = length(test)

In [10]:
t = train[train['head_tail']!=0]
d = dev[dev['head_tail']!=0]
# split from sender
train_c = t[t.sender=='customer']
train_h = t[t.sender=='helpdesk']
dev_c = d[d.sender=='customer']
dev_h = d[d.sender=='helpdesk']

In [11]:
train.head()

,id,sender,sender_num,texts,round,CNUG0,CNUG,CNUG*,CNaN,HNUG,...,HNaN,W2V,W2V_str,Baidu,Baidu_str,index,round_feature,head_tail,length,accumulate
0,3830401296796826,customer,0,中国电信 控制箱 没有 维护 信息安全 人身安全 保障 好好 修修 应该 差钱 位于 济南市...,1,0.736842,0.052632,0.0,0.210526,0.000000,...,0.000000,"[0.082410949456, 0.041056426199999996, -0.1564...",0.082410949456 0.041056426199999996 -0.15647...,"[-0.004204561904761913, -0.14198089523809523, ...",-0.004204561904761913 -0.14198089523809523 0.0...,0,"[1, 2]",0,64,"[0.082410949456, 0.041056426199999996, -0.1564..."
1,3830401296796826,helpdesk,1,您好 反映 情况 认真 记录 会 及时 相关 部门 反馈 敬请 等待,2,0.000000,0.000000,0.0,0.000000,0.473684,...,0.526316,"[-0.03553892808333333, 0.17461223858333333, -0...",-0.03553892808333333 0.17461223858333333 -0....,"[-0.05267071666666667, 0.09319805000000002, 0....",-0.05267071666666667 0.09319805000000002 0.087...,1,"[2, 2]",2,23,"[0.023436010686333334, 0.10783433239166666, -0..."
2,3830772740080373,customer,0,电信服务 广州市 白云区 太和 镇 谢家 庄 二队 电信 信号 差 投诉 几年 没人 处理 ...,1,0.736842,0.000000,0.0,0.263158,0.000000,...,0.000000,"[0.01378158932432433, 0.15494613002702703, -0....",0.01378158932432433 0.15494613002702703 -0.1...,"[0.022850767647058803, 0.00045413235294118045,...",0.022850767647058803 0.00045413235294118045 -0...,2,"[1, 6]",0,85,"[0.01378158932432433, 0.15494613002702703, -0...."
3,3830772740080373,helpdesk,1,您好 中国电信 广东 客服 关注 反映 问题 请 详细描述 一下 问题 处理 谢谢,2,0.000000,0.000000,0.0,0.000000,0.526316,...,0.421053,"[-0.030600412399999992, 0.07297016430769229, -...",-0.030600412399999992 0.07297016430769229 -0...,"[-0.36042518181818184, 0.18539584545454543, 0....",-0.36042518181818184 0.18539584545454543 0.066...,3,"[2, 6]",1,29,"[-0.008409411537837832, 0.11395814716735966, -..."
4,3830772740080373,customer,0,图片 知道 起码 通信 信号 没有 更 3G 上网 信号 一年 前 两年 前 第一次 电话 ...,3,0.052632,0.736842,0.0,0.210526,0.000000,...,0.000000,"[0.009741488182051285, 0.19042638691025635, -0...",0.009741488182051285 0.19042638691025635 -0....,"[-0.05475728378378378, 0.016655232432432426, 0...",-0.05475728378378378 0.016655232432432426 0.04...,4,"[3, 6]",1,80,"[-0.010429462108974354, 0.13169827560897432, -..."


In [217]:
# === customer
c_X_train = train_c.filter(['round','length','texts','round_feature'])
c_X_test = dev_c.filter(['round','length','texts','round_feature'])
    

y_train_c = train_c.filter(['CNUG0','CNUG','CNUG*','CNaN']).values
y_test_c = dev_c.filter(['CNUG0','CNUG','CNUG*','CNaN']).values

# y_train_h = train.filter(['HNUG','HNUG*','HNaN'])
# y_test_h = dev.filter(['HNUG','HNUG*','HNaN'])

c_X1_train = c_X_train['texts']
# c_X1_train = [str (item) for item in c_X1_train]
c_X1_test = c_X_test['texts']
# c_X1_test = [str (item) for item in c_X1_test]

c_X2_train = c_X_train[['round','length']].values
c_X2_test = c_X_test[['round','length']].values

c_X3_train = np.array(c_X_train['round_feature'].values.tolist())
c_X3_test = np.array(c_X_test['round_feature'].values.tolist())

c_token = Tokenizer(num_words = 20000)
c_token.fit_on_texts(c_X1_train)
c_vocab = c_token.word_index
print(c_token.document_count)

c_x_train_seq = c_token.texts_to_sequences(c_X1_train)
c_x_test_seq = c_token.texts_to_sequences(c_X1_test)
c_X1_train = sequence.pad_sequences(c_x_train_seq, maxlen = 350)
c_X1_test = sequence.pad_sequences(c_x_test_seq, maxlen = 350)

# === helpdesk
h_X_train = train_h.filter(['round','length','texts','round_feature'])
h_X_test = dev_h.filter(['round','length','texts','round_feature'])
    

# y_train_c = train_c.filter(['CNUG','CNUG*','CNUG0','CNaN'])
# y_test_c = dev_c.filter(['CNUG','CNUG*','CNUG0','CNaN'])

y_train_h = train_h.filter(['HNUG','HNUG*','HNaN'])
y_test_h = dev_h.filter(['HNUG','HNUG*','HNaN'])

h_X1_train = h_X_train['texts']
# h_X1_train = [str (item) for item in h_X1_train]
h_X1_test = h_X_test['texts']
# h_X1_test = [str (item) for item in h_X1_test]

h_X2_train = h_X_train[['round','length']].values
h_X2_test = h_X_test[['round','length']].values

h_X3_train = np.array(h_X_train['round_feature'].values.tolist())
h_X3_test = np.array(h_X_test['round_feature'].values.tolist())

h_token = Tokenizer(num_words = 20000)
h_token.fit_on_texts(h_X1_train)
h_vocab = h_token.word_index
print(h_token.document_count)

h_x_train_seq = h_token.texts_to_sequences(h_X1_train)
h_x_test_seq = h_token.texts_to_sequences(h_X1_test)
h_X1_train = sequence.pad_sequences(h_x_train_seq, maxlen = 350)
h_X1_test = sequence.pad_sequences(h_x_test_seq, maxlen = 350)

4800
6900


In [96]:
def to_array(dataframe, length, string, dim=256):   
    train_vec = np.zeros(shape=(length,dim))
#     print(train_vec.shape)
    for i in range(len(dataframe[string])):
        if dataframe[string].iloc[i] != []:
#             print(type(dataframe[string].iloc[i]))
    #         print(i)
            train_vec[i] = dataframe[string].iloc[i]
        else:
            continue
    print(train_vec.shape)
    
    return train_vec

train_vec_c = to_array(train_c, len(train_c), 'W2V', dim=300)
train_vec_h = to_array(train_h, len(train_h), 'W2V', dim=300)
dev_vec_c = to_array(dev_c, len(dev_c), 'W2V', dim=300)
dev_vec_h = to_array(dev_h, len(dev_h), 'W2V', dim=300)
# test_vec_c = to_array(test_c, len(test_c), 'W2V', dim=300)
# test_vec_h = to_array(test_h, len(test_h), 'W2V', dim=300)

train_vec_c = np.reshape(train_vec_c, (train_vec_c.shape[0],1, train_vec_c.shape[1]))
train_vec_h = np.reshape(train_vec_h, (train_vec_h.shape[0],1, train_vec_h.shape[1]))
dev_vec_c = np.reshape(dev_vec_c, (dev_vec_c.shape[0],1,dev_vec_c.shape[1]))
dev_vec_h = np.reshape(dev_vec_h, (dev_vec_h.shape[0],1,dev_vec_h.shape[1]))
# test_vec_c = np.reshape(test_vec_c, (test_vec_c.shape[0],1,test_vec_c.shape[1]))
# test_vec_h = np.reshape(test_vec_h, (test_vec_h.shape[0],1,test_vec_h.shape[1]))

(4800, 300)
(6900, 300)
(585, 300)
(780, 300)


In [223]:
def lstm_C(X1_train, X2_train, X3_train, X1_test, 
           X2_test, X3_test, y_train, y_test, loss='categorical_crossentropy'):
     
#     main_input = Input(shape=(350,), dtype='float64')
    main_input = Input(shape=(1,300))
    sub_input = Input(shape=(2,))
    tri_input = Input(shape=(2,))
    
#     embed = Embedding(output_dim=300,input_dim=20000,input_length=350)(main_input)
#     dropout_1 = Dropout(0.35)(embed)
    lst = Bidirectional(LSTM(units=256,return_sequences=True))(main_input)
    lst2 = Bidirectional(LSTM(units=128))(lst)
    merge = concatenate([lst2, sub_input, tri_input])
    dense_1 = Dense(units=64,activation='relu')(merge)
    dropout_1 = Dropout(0.35)(dense_1)
#     dense_2 = Dense(units=32,activation='relu')(dropout_1)
    dense_3 = Dense(units=32,activation='relu')(dropout_1)
    output = Dense(units=4,activation='softmax')(dense_3)

    model = Model(inputs=[main_input, sub_input, tri_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    filepath="C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Model file\\weights(h).best.hdf5"
    checkpoint= ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                                save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    
    train_history = model.fit(x=[X1_train, X2_train, X3_train], y=y_train, epochs=50, 
                              batch_size=128, callbacks=callbacks_list, verbose=1, validation_split=0.2)

    score = model.evaluate(x=[X1_test, X2_test, X3_test], y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=[X1_test, X2_test, X3_test])
    predicted = pre_probability.argmax(axis=-1)
    
    return model, train_history, pre_probability


# === helpdesk
def lstm_H(X1_train, X2_train, X3_train, X1_test, X2_test, 
           X3_test, y_train, y_test, loss='categorical_crossentropy'):

#     main_input = Input(shape=(350,), dtype='float64')
    main_input = Input(shape=(1,300))
    sub_input = Input(shape=(2,))
    tri_input = Input(shape=(2,))
    
#     embed = Embedding(output_dim=300,input_dim=20000,input_length=350)(main_input)
#     dropout_1 = Dropout(0.35)(embed)
    lst = Bidirectional(LSTM(units=256,return_sequences=True))(main_input)
    lst2 = Bidirectional(LSTM(units=128))(lst)
    merge = concatenate([lst2, sub_input, tri_input])
    dense_1 = Dense(units=64,activation='relu')(merge)
    dropout_1 = Dropout(0.35)(dense_1)
#     dense_2 = Dense(units=32,activation='relu')(dropout_1)
    dense_3 = Dense(units=32,activation='relu')(dropout_1)
    output = Dense(units=3,activation='softmax')(dense_3)

    model = Model(inputs=[main_input, sub_input, tri_input], outputs=output)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    print(model.summary())

    # checkpoint
    filepath="C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Model file\\weights(h).best.hdf5"
    checkpoint= ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                                save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    
    train_history = model.fit(x=[X1_train, X2_train, X3_train], y=y_train, epochs=50, 
                              batch_size=128, callbacks=callbacks_list, verbose=1, validation_split=0.2)

    score = model.evaluate(x=[X1_test, X2_test, X3_test], y=y_test, verbose=1)

    print("Test Score:", score[0])
    print("Test Accuracy:", score[1])

    pre_probability = model.predict(x=[X1_test, X2_test, X3_test])
    predicted = pre_probability.argmax(axis=-1)
    
    return model, train_history, pre_probability
    

In [216]:
lstm_c_model, lstm_c_history, lstm_c_pred = lstm_C(train_vec_c, c_X2_train, c_X3_train, dev_vec_c, 
                                                   c_X2_test, c_X3_test, y_train_c, y_test_c, 
                                                   loss = jsd_custom_loss)

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_38 (InputLayer)           [(None, 1, 300)]     0                                            
__________________________________________________________________________________________________
bidirectional_30 (Bidirectional (None, 1, 512)       1140736     input_38[0][0]                   
__________________________________________________________________________________________________
bidirectional_31 (Bidirectional (None, 256)          656384      bidirectional_30[0][0]           
__________________________________________________________________________________________________
input_39 (InputLayer)           [(None, 2)]          0                                            
___________________________________________________________________________________________

Epoch 20/50
3712/3840 [============================>.] - ETA: 0s - loss: 0.0346 - acc: 0.8276
Epoch 00020: val_acc improved from 0.83333 to 0.83542, saving model to C:\Users\doudi\OneDrive\Documents\ntcir15\Model file\weights(h).best.hdf5
3840/3840 [==============================] - 1s 135us/sample - loss: 0.0346 - acc: 0.8263 - val_loss: 0.0369 - val_acc: 0.8354
Epoch 21/50
3328/3840 [=========================>....] - ETA: 0s - loss: 0.0340 - acc: 0.8290
Epoch 00021: val_acc did not improve from 0.83542
3840/3840 [==============================] - 0s 116us/sample - loss: 0.0344 - acc: 0.8281 - val_loss: 0.0398 - val_acc: 0.8260
Epoch 22/50
3584/3840 [===========================>..] - ETA: 0s - loss: 0.0345 - acc: 0.8234
Epoch 00022: val_acc improved from 0.83542 to 0.83854, saving model to C:\Users\doudi\OneDrive\Documents\ntcir15\Model file\weights(h).best.hdf5
3840/3840 [==============================] - 0s 125us/sample - loss: 0.0347 - acc: 0.8219 - val_loss: 0.0396 - val_acc: 0.83

Epoch 50/50
3712/3840 [============================>.] - ETA: 0s - loss: 0.0266 - acc: 0.8596
Epoch 00050: val_acc did not improve from 0.83854
585/585 [==============================] - 0s 165us/sample - loss: 0.0353 - acc: 0.8103
Test Score: 0.035299732696869944
Test Accuracy: 0.8102564


In [224]:
lstm_h_model, lstm_h_history, lstm_h_pred = lstm_H(train_vec_h, h_X2_train, h_X3_train, dev_vec_h, 
                                                   h_X2_test, h_X3_test, y_train_h, y_test_h, 
                                                   loss = jsd_custom_loss)

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41 (InputLayer)           [(None, 1, 300)]     0                                            
__________________________________________________________________________________________________
bidirectional_32 (Bidirectional (None, 1, 512)       1140736     input_41[0][0]                   
__________________________________________________________________________________________________
bidirectional_33 (Bidirectional (None, 256)          656384      bidirectional_32[0][0]           
__________________________________________________________________________________________________
input_42 (InputLayer)           [(None, 2)]          0                                            
___________________________________________________________________________________________

Epoch 19/50
5248/5520 [===========================>..] - ETA: 0s - loss: 0.0211 - acc: 0.8740
Epoch 00019: val_acc did not improve from 0.88043
5520/5520 [==============================] - 1s 121us/sample - loss: 0.0209 - acc: 0.8755 - val_loss: 0.0221 - val_acc: 0.8790
Epoch 20/50
5120/5520 [==========================>...] - ETA: 0s - loss: 0.0206 - acc: 0.8793
Epoch 00020: val_acc did not improve from 0.88043
5520/5520 [==============================] - 1s 116us/sample - loss: 0.0206 - acc: 0.8774 - val_loss: 0.0223 - val_acc: 0.8790
Epoch 21/50
5376/5520 [============================>.] - ETA: 0s - loss: 0.0206 - acc: 0.8782
Epoch 00021: val_acc improved from 0.88043 to 0.88116, saving model to C:\Users\doudi\OneDrive\Documents\ntcir15\Model file\weights(h).best.hdf5
5520/5520 [==============================] - 1s 128us/sample - loss: 0.0206 - acc: 0.8757 - val_loss: 0.0215 - val_acc: 0.8812
Epoch 22/50
5376/5520 [============================>.] - ETA: 0s - loss: 0.0200 - acc: 0.878

numpy.float32

In [275]:
output_c = pd.DataFrame(list(zip(d[d.sender=='customer']['index'].tolist(), lstm_c_pred.tolist())), 
                        columns =['Index', 'label'])
output_h = pd.DataFrame(list(zip(d[d.sender=='helpdesk']['index'].tolist(), lstm_h_pred.tolist())), 
                        columns =['Index', 'label'])

In [286]:
from itertools import chain

d2 = dev[dev['head_tail']==0]

temp = train[train['head_tail']==0].groupby('sender').mean()
tmp = temp[['CNUG0','CNUG','CNUG*','CNaN']].values.tolist()
# tmp = [[1,0,0,0]]

t = []
for i in range(len(d2)):
    t.append(list(chain(*tmp)))
    
    
output_first_round = pd.DataFrame(list(zip(d2['index'].tolist(), t)),
                                  columns =['Index', 'label'])

In [287]:
output = pd.concat([output_c,output_h,output_first_round], ignore_index=True)  
output = output.sort_values(by=['Index'])
output = output.reset_index(drop=True)

In [288]:
output['id'] = dev['id'].apply(str)
output['sender'] = dev['sender']


In [289]:
len(output)

1755

In [290]:
def Generate_submission(dev):
    dev['id'] = dev['id'].apply(str)
    Id_list = dev['id'].unique()
    C_nugget = ['CNUG0','CNUG','CNUG*','CNaN']
    H_nugget = ['HNUG','HNUG*','HNaN']

    final = []
    
    # go through each Id first
    for Id in tqdm(Id_list):  
        result = []
        
        for i in range(len(dev)):
            # if Id is match than predict the prob_distribution and zip it as dictionary 
            if dev['id'][i] == Id:
                if dev.iloc[i, 3] == 'customer':
                    cus_prob = dev['label'][i]
                    dict_c = dict(zip(C_nugget, cus_prob))
                    result.append(dict_c)
                else:
                    help_prob = dev['label'][i]
                    dict_h = dict(zip(H_nugget, help_prob))
                    result.append(dict_h)
            else:
                continue
        
        # Submission form
        dict1 = {'nugget':result,'id':Id}
        final.append(dict1)
        
    return final

In [291]:
final = Generate_submission(output)

C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [268]:
final.item()

AttributeError: 'list' object has no attribute 'item'

In [292]:
import json
import time
import os

path = 'C:/Users/doudi/OneDrive/Documents/ntcir15/eval'
os.chdir(path)
timestr = time.strftime("%Y%m%d%H%M")
with open((timestr + '_' + 'dev_eval.json'), 'w', encoding='utf-8') as f: 
    f.write(json.dumps(final, ensure_ascii=False, indent=2))

In [293]:
import os
os.chdir('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\eval')
!python eval.py 202007211820_dev_eval.json dev_cn.json

{'quality': None, 'nugget': {'jsd': 4.208808998831172, 'rnss': 2.859064105686865}}


In [294]:
def cal(num):
    return 2**(-num)

cal(4.208808998831172)

0.0540782022047612

### 3. 訓練各輪次時也要放入前1輪的對話內容

In [308]:
def accumulate(dataframe):
    final = []
    # go through each Id first 
    for i in tqdm(range(len(dataframe))):
#                 length = len(dataframe[dataframe['id']==Id])
        if dataframe['round'][i] == 1:
            final.append(dataframe['W2V'][i])
        else:
            t = [(x+y)/2 for x,y in zip(dataframe['W2V'][i],dataframe['W2V'][i-1])]
            final.append(t)
    return final

In [309]:
train['accumulate'] = accumulate(train)
dev['accumulate'] = accumulate(dev)

C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [311]:
t = train[train['head_tail']!=0]
d = dev[dev['head_tail']!=0]
# split from sender
train_c = t[t.sender=='customer']
train_h = t[t.sender=='helpdesk']
dev_c = d[d.sender=='customer']
dev_h = d[d.sender=='helpdesk']

def to_array(dataframe, length, string, dim=256):   
    train_vec = np.zeros(shape=(length,dim))
#     print(train_vec.shape)
    for i in range(len(dataframe[string])):
        if dataframe[string].iloc[i] != []:
#             print(type(dataframe[string].iloc[i]))
    #         print(i)
            train_vec[i] = dataframe[string].iloc[i]
        else:
            continue
    print(train_vec.shape)
    
    return train_vec

train_vec_c = to_array(train_c, len(train_c), 'accumulate', dim=300)
train_vec_h = to_array(train_h, len(train_h), 'accumulate', dim=300)
dev_vec_c = to_array(dev_c, len(dev_c), 'accumulate', dim=300)
dev_vec_h = to_array(dev_h, len(dev_h), 'accumulate', dim=300)
# test_vec_c = to_array(test_c, len(test_c), 'W2V', dim=300)
# test_vec_h = to_array(test_h, len(test_h), 'W2V', dim=300)

train_vec_c = np.reshape(train_vec_c, (train_vec_c.shape[0],1, train_vec_c.shape[1]))
train_vec_h = np.reshape(train_vec_h, (train_vec_h.shape[0],1, train_vec_h.shape[1]))
dev_vec_c = np.reshape(dev_vec_c, (dev_vec_c.shape[0],1,dev_vec_c.shape[1]))
dev_vec_h = np.reshape(dev_vec_h, (dev_vec_h.shape[0],1,dev_vec_h.shape[1]))
# test_vec_c = np.reshape(test_vec_c, (test_vec_c.shape[0],1,test_vec_c.shape[1]))
# test_vec_h = np.reshape(test_vec_h, (test_vec_h.shape[0],1,test_vec_h.shape[1]))

(4800, 300)
(6900, 300)
(585, 300)
(780, 300)


In [312]:
lstm_c_model, lstm_c_history, lstm_c_pred = lstm_C(train_vec_c, c_X2_train, c_X3_train, dev_vec_c, 
                                                   c_X2_test, c_X3_test, y_train_c, y_test_c, 
                                                   loss = jsd_custom_loss)

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           [(None, 1, 300)]     0                                            
__________________________________________________________________________________________________
bidirectional_34 (Bidirectional (None, 1, 512)       1140736     input_44[0][0]                   
__________________________________________________________________________________________________
bidirectional_35 (Bidirectional (None, 256)          656384      bidirectional_34[0][0]           
__________________________________________________________________________________________________
input_45 (InputLayer)           [(None, 2)]          0                                            
___________________________________________________________________________________________

Epoch 19/50
3712/3840 [============================>.] - ETA: 0s - loss: 0.0342 - acc: 0.8362
Epoch 00019: val_acc improved from 0.84583 to 0.85417, saving model to C:\Users\doudi\OneDrive\Documents\ntcir15\Model file\weights(h).best.hdf5
3840/3840 [==============================] - 1s 141us/sample - loss: 0.0342 - acc: 0.8370 - val_loss: 0.0353 - val_acc: 0.8542
Epoch 20/50
3712/3840 [============================>.] - ETA: 0s - loss: 0.0337 - acc: 0.8359
Epoch 00020: val_acc did not improve from 0.85417
3840/3840 [==============================] - 0s 126us/sample - loss: 0.0339 - acc: 0.8344 - val_loss: 0.0364 - val_acc: 0.8500
Epoch 21/50
3712/3840 [============================>.] - ETA: 0s - loss: 0.0336 - acc: 0.8424
Epoch 00021: val_acc did not improve from 0.85417
3840/3840 [==============================] - 0s 127us/sample - loss: 0.0335 - acc: 0.8417 - val_loss: 0.0358 - val_acc: 0.8500
Epoch 22/50
3712/3840 [============================>.] - ETA: 0s - loss: 0.0335 - acc: 0.841

Epoch 49/50
3584/3840 [===========================>..] - ETA: 0s - loss: 0.0275 - acc: 0.8655
Epoch 00049: val_acc did not improve from 0.85417
3840/3840 [==============================] - 0s 125us/sample - loss: 0.0276 - acc: 0.8656 - val_loss: 0.0362 - val_acc: 0.8406
Epoch 50/50
3712/3840 [============================>.] - ETA: 0s - loss: 0.0263 - acc: 0.8734
Epoch 00050: val_acc did not improve from 0.85417
585/585 [==============================] - 0s 162us/sample - loss: 0.0358 - acc: 0.8120
Test Score: 0.035760275569035965
Test Accuracy: 0.8119658


In [313]:
lstm_h_model, lstm_h_history, lstm_h_pred = lstm_H(train_vec_h, h_X2_train, h_X3_train, dev_vec_h, 
                                                   h_X2_test, h_X3_test, y_train_h, y_test_h, 
                                                   loss = jsd_custom_loss)

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_47 (InputLayer)           [(None, 1, 300)]     0                                            
__________________________________________________________________________________________________
bidirectional_36 (Bidirectional (None, 1, 512)       1140736     input_47[0][0]                   
__________________________________________________________________________________________________
bidirectional_37 (Bidirectional (None, 256)          656384      bidirectional_36[0][0]           
__________________________________________________________________________________________________
input_48 (InputLayer)           [(None, 2)]          0                                            
___________________________________________________________________________________________

Epoch 48/50
5248/5520 [===========================>..] - ETA: 0s - loss: 0.0176 - acc: 0.8931
Epoch 00048: val_acc did not improve from 0.88261
5520/5520 [==============================] - 1s 119us/sample - loss: 0.0177 - acc: 0.8929 - val_loss: 0.0229 - val_acc: 0.8746
Epoch 49/50
5248/5520 [===========================>..] - ETA: 0s - loss: 0.0173 - acc: 0.8880
Epoch 00049: val_acc did not improve from 0.88261
5520/5520 [==============================] - 1s 121us/sample - loss: 0.0175 - acc: 0.8866 - val_loss: 0.0227 - val_acc: 0.8775
Epoch 50/50
5248/5520 [===========================>..] - ETA: 0s - loss: 0.0171 - acc: 0.8883
Epoch 00050: val_acc did not improve from 0.88261
780/780 [==============================] - 0s 169us/sample - loss: 0.0230 - acc: 0.8705
Test Score: 0.022966900372352356
Test Accuracy: 0.87051284


In [314]:
output_c = pd.DataFrame(list(zip(d[d.sender=='customer']['index'].tolist(), lstm_c_pred.tolist())), 
                        columns =['Index', 'label'])
output_h = pd.DataFrame(list(zip(d[d.sender=='helpdesk']['index'].tolist(), lstm_h_pred.tolist())), 
                        columns =['Index', 'label'])
from itertools import chain

d2 = dev[dev['head_tail']==0]

temp = train[train['head_tail']==0].groupby('sender').mean()
tmp = temp[['CNUG0','CNUG','CNUG*','CNaN']].values.tolist()
# tmp = [[1,0,0,0]]

t = []
for i in range(len(d2)):
    t.append(list(chain(*tmp)))
    
    
output_first_round = pd.DataFrame(list(zip(d2['index'].tolist(), t)),
                                  columns =['Index', 'label'])

output = pd.concat([output_c,output_h,output_first_round], ignore_index=True)  
output = output.sort_values(by=['Index'])
output = output.reset_index(drop=True)
output['id'] = dev['id'].apply(str)
output['sender'] = dev['sender']
print(len(output))

1755


In [315]:
final = Generate_submission(output)

C:\Users\doudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


In [316]:
import json
import time
import os

path = 'C:/Users/doudi/OneDrive/Documents/ntcir15/eval'
os.chdir(path)
timestr = time.strftime("%Y%m%d%H%M")
with open((timestr + '_' + 'dev_eval.json'), 'w', encoding='utf-8') as f: 
    f.write(json.dumps(final, ensure_ascii=False, indent=2))

In [317]:
import os
os.chdir('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\eval')
!python eval.py 202007211902_dev_eval.json dev_cn.json

{'quality': None, 'nugget': {'jsd': 4.731563183129878, 'rnss': 3.041679084438695}}


In [319]:
def cal(num):
    return 2**(-num)

cal(3.041679084438695)

0.12144044736003629

In [320]:
train.to_csv('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\train_feature.csv', 
             index=False, encoding='utf_8_sig')
dev.to_csv('C:\\Users\\doudi\\OneDrive\\Documents\\ntcir15\\Dataset\\New_DialEval-1\\dev_feature.csv', 
           index=False, encoding='utf_8_sig')